# Загрузка библиотек

In [1]:
import os
import cv2
import yaml
import shutil
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from tqdm import tqdm
from pathlib import Path

# Создание аугментированных изображений для train

In [2]:
transform_spatter = {
    'spatter':
    A.Compose([
        A.Spatter(always_apply=True, intensity=0.2)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_crop = {
    'crop':
    A.Compose([
    A.RandomCrop(height=300, width=300, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_resizedcrop = {
    'resizedcrop':
    A.Compose([
    A.RandomResizedCrop(height=300, width=300, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_snow = {
    'snow':
    A.Compose([
    A.RandomSnow(always_apply=True, p=1, brightness_coeff=1.6)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_rain = {
    'rain':
    A.Compose([
    A.RandomRain(always_apply=True, p=1, blur_value=1, brightness_coefficient=0.8)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_shadow = {
    'shadow':
    A.Compose([
    A.RandomShadow(always_apply=True, p=1)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_jitter = {
    'jitter':
    A.Compose([
    A.ColorJitter(always_apply=True, p=1),
    A.RandomCrop(500, 500, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_hue = {
    'hue':
    A.Compose([
    A.HueSaturationValue(always_apply=True, p=1),
    A.RandomCrop(500, 500, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_emboss = {
    'emboss':
    A.Compose([
    A.Emboss(always_apply=True, p=1),
    A.RandomCrop(500, 500, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

transform_rgb = {
    'rgb':
    A.Compose([
    A.RGBShift(always_apply=True, p=1),
    A.RandomCrop(500, 500, always_apply=True),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'])
)}

In [3]:
transforms = [
    transform_spatter,
    transform_crop,
    transform_resizedcrop,
    transform_snow,
    transform_rain,
    transform_shadow,
    transform_jitter,
    transform_hue,
    transform_emboss,
    transform_rgb
]

In [4]:
def get_transformed_data(image, bboxes, transform):
    transformed = transform(
        image=image, bboxes=bboxes[:, :-1].astype(float),
        class_labels=bboxes[:, -1].astype(int)
    )
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    transformed_labels = transformed['class_labels']
    transformed_bboxes = np.append(
        np.array(transformed_labels).reshape(-1, 1), 
        np.array(transformed_bboxes), axis=1
    )
    return transformed_image, transformed_bboxes

In [5]:
image_path = Path('train\images')
label_path = Path('train\labels')

images = os.listdir(image_path) 
np.random.shuffle(images)
for image_name in tqdm(images):
    # skip augmented photos
    if np.any([x in image_name for x in [list(trans.keys())[0] for trans in transforms]]):
        continue
    image = cv2.imread((image_path / image_name).__str__())
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    common_file_name = image_name.split('.jpg')[0]

    bbox_path = common_file_name + '.txt'
    bbox_path = label_path / bbox_path
    with open(bbox_path, 'r') as file:
        bboxes = file.readlines()
    if len(bboxes)==0:
        continue
    bboxes = [bbox.replace('\n', '').split(' ') for bbox in bboxes]
    bboxes = np.append(
        np.array(bboxes)[:, 1:], 
        np.array(bboxes)[:, 0].reshape(-1, 1), 
        axis=1
    )

    for transform_dict in transforms:
        key = list(transform_dict.keys())[0]
        transform = list(transform_dict.values())[0]
        # transformed_image, transformed_bboxes = get_transformed_data(image, bboxes, transform)
        transformed = transform(
            image=image, bboxes=bboxes[:, :-1].astype(float),
            class_labels=bboxes[:, -1].astype(int)
        )
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_labels = transformed['class_labels']
        labs = np.array(transformed_labels).reshape(-1, 1)
        bbs = np.array(transformed_bboxes)
        if labs.shape[0]==0 or bbs.shape[0]==0:
            continue
        transformed_bboxes = np.append(
            labs, bbs, axis=1
        ) 
        im = Image.fromarray(transformed_image)
        im.save((image_path / common_file_name).__str__()+f'_{key}.jpg')

        with open((label_path / common_file_name).__str__()+f'_{key}.txt', 'w') as f:
            for i, box in enumerate(transformed_bboxes):
                row = box[0].astype(int).astype(str)+' ' + ' '.join(box[1:].round(8).astype(str))
                f.write(row)
                if i != len(transformed_bboxes):
                    f.write('\n')

100%|██████████| 885/885 [02:11<00:00,  6.73it/s]
